In [1]:
from __future__ import print_function, division
from builtins import range

In [2]:
import gym
import os
import sys
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from gym import wrappers
from datetime import datetime
from scipy.misc import imresize

ImportError: cannot import name 'is_string_like'

In [ ]:
IM_SIZE = 24
IM_SIZE = 24

In [4]:
CUDA_VISIBLE_DEVICES=0 #Using Nvidia GPU:0
TARGET_UPDATE_PERIOD = 100
MAX_EXPERIENCES = 500000
MIN_EXPERIENCES = 50000
K = 6 #action space

In [5]:
def downsample_image(A):
    B = A[31:195]
    B = B.mean(axis =2)
    B = B/255.0
    B = imresize(B, size= (IM_SIZE, IM_SIZE), interp= 'nearest')
    return B

In [6]:
tf.reset_default_graph()

In [7]:
class DQN:
    def __init__(self, K, conv_layer_sizes, hidden_layer_sizes, gamma, scope):
        
        self.K= K
        self.scope = scope
        
        with tf.variable_scope(scope):
            #considering input as 4 series of images
            self.X = tf.placeholder(tf.float32, shape = (None, 4, IM_SIZE, IM_SIZE), name = 'X') 
            #order: (num_samples, height, width, "color")
            
            #RL variables
            self.G = tf.placeholder(tf.float32, shape = (None, ), name = 'G')
            self.actions = tf.placeholder(tf.int32, shape = (None, ), name = 'actions')
            
            #convolution
            Z =self.X/255.0
            Z= tf.transpose(Z, [0,2,3,1])
            
            for num_output_filters, filtersz, stridesz in conv_layer_sizes:
                #print("debugging: ")
                #print((num_output_filters, filtersz, stridesz))
                Z = tf.contrib.layers.conv2d(Z, num_output_filters, filtersz, stride = stridesz, activation_fn=tf.nn.relu)
                
            #fully connected layers
            Z = tf.contrib.layers.flatten(Z)
            for M in hidden_layer_sizes:
                Z = tf.contrib.layers.fully_connected(Z, M)
            
            #final layer
            self.predict_op = tf.contrib.layers.fully_connected(Z, K)
            
            #also one_hot_encode_all_predictions(actions)
            selected_action_values = tf.reduce_sum(self.predict_op* tf.one_hot(self.actions, self.K), reduction_indices=[1])
            
            cost = tf.reduce_sum(tf.square(self.G- selected_action_values))
                
            self.train_op = tf.train.RMSPropOptimizer(0.00025, 0.99, 0.0, 1e-6).minimize(cost)
     
            self.cost = cost
    
    def copy_from(self, other):
        mine = [t for t in tf.trainable_variables() if t.name.startswith(self.scope)]
        mine = sorted(mine, key= lambda x:x.name)
        theirs = [t for t in tf.trainable_variables() if t.name.startswith(other.scope)]
        theirs = sorted(theirs, key=lambda x:x.name)

        ops = []
        for p, q in zip(mine, theirs):
            actual = self.session.run(q)
            op = p.assign(actual)
            ops.append(op)
        self.session.run(ops)
    
    def set_session(self, session):
        self.session = session
        
    def predict(self, states):
        return self.session.run(self.predict_op, feed_dict = {self.X: states})
    
    def update(self, states, actions, targets):
        c,_ = self.session.run([self.cost, self.train_op], feed_dict = {self.X:states, self.G:targets, self.actions:actions})
        return c
    
    def sample_action(self, x, eps):
        if np.random.random() < eps:
            return np.random.choice(self.K)
        else:
            return np.argmax(self.predict([x])[0])

In [8]:
def learn(model, target_model, experience_replay_buffer, gamma, batch_size):
    #Sample experiences:
    samples = random.sample(experience_replay_buffer, batch_size)
    states, actions, rewards, next_states, dones = map(np.array, zip(*samples))

    # Calculate the targets
    next_Qs= target_model.predict(next_states)

    #the reward with max reward:
    next_Q = np.max(next_Qs, axis = 1)

    #not including future state if the game is over
    targets = rewards + np.invert(dones).astype(np.float32)*gamma*(next_Q)

    #Update the model
    #Here current model is the learnig agent, target model is the temporarily stable one taking rest
    loss = model.update(states, actions, targets)
    return loss

In [9]:
def play_one(env, total_t, experience_replay_buffer, model, target_model, gamma, batch_size, epsilon, epsilon_change, epsilon_min):
    t0 = datetime.now()

    #Reset the environment
    obs = env.reset()
    obs_small = downsample_image(obs)
    #always state is most recent 4 frames
    state = np.stack([obs_small]*4, axis =0)
    assert(state.shape == (4, 80, 80))
    loss = None
        
    total_time_training = 0
    num_steps_in_episode = 1
    episode_reward = 0
    #env = env.monitor.start('../neural_reinforcement_agents')
    done = False
    while not done:
        #Update the current training network

        if total_t % TARGET_UPDATE_PERIOD == 0:
            #periodically save the current learnings into a temp copy to bring extra stability
            target_model.copy_from(model)

        #also take actions to learn the game
        action = model.sample_action(state, epsilon)

        #find the reward
        obs, reward, done,_ = env.step(action)    
        obs_small = downsample_image(obs)

        next_state = np.append(state[1:],  np.expand_dims(obs_small, axis = 0), axis = 0)
        state  = next_state



        total_t += 1
        episode_reward += reward
        num_steps_in_episode += num_steps_in_episode


        #updating the experience replay buffer
        if len(experience_replay_buffer)> MAX_EXPERIENCES:
            experience_replay_buffer.pop(0)

        experience_replay_buffer.append((state, action, reward, next_state, done))

        #train the model
        t0_2 = datetime.now()
        
        loss = learn(model, target_model, experience_replay_buffer, gamma, batch_size)
        
        dt = datetime.now() - t0_2

        total_time_training += dt.total_seconds()

        #updating the epsilon value
        epsilon = max(epsilon- epsilon_change, epsilon_min)

    return total_t, episode_reward, (datetime.now() - t0), num_steps_in_episode, epsilon

In [10]:
def update_state(state, obs):
    obs_small = downsample_image(obs)
    return np.append(state[1:], np.expand_dims(obs_small, 0), axis=0)

In [11]:
if __name__ == '__main__':    
    #hyperparameters and initialization
    conv_layer_sizes = [(32, 8, 4), (64, 4, 2), (64, 3, 1)]
    hidden_layer_sizes = [512]
    gamma = 0.99
    batch_size = 32
    num_episodes= 10
    total_t = 0
    experience_replay_buffer = []
    episode_rewards = np.zeros(num_episodes)

    
    #epsilon decays over time
    epsilon = 1.0
    epsilon_min = 0.1
    epsilon_change = (epsilon - epsilon_min)/ 500000

    #Make the environment
    env = gym.envs.make("Assault-v0")
    #env = wrappers.Monitor(env, '../neural_reinforcement_agents')

    #Creat models
    model = DQN(K= K, conv_layer_sizes = conv_layer_sizes, hidden_layer_sizes = hidden_layer_sizes, gamma= gamma, scope = "model")

    target_model = DQN(K=K, conv_layer_sizes = conv_layer_sizes, hidden_layer_sizes = hidden_layer_sizes, gamma= gamma, scope = "target_model")
    
    with tf.Session() as sess:

        #Trying to fill in the experience buffer not learning anything!
        model.set_session(sess)
        target_model.set_session(sess)

        sess.run(tf.global_variables_initializer())

        obs = env.reset()
        obs_small = downsample_image(obs)

        state = np.stack([obs_small]*4, axis =0)

        for i in range(MIN_EXPERIENCES):
            action = np.random.choice(K)
            print('Experience: ', i)
            
            obs, reward, done,_ = env.step(action)
            env.render()
            next_state = update_state(state, obs)

            experience_replay_buffer.append((state, action, reward, next_state, done))

            if done:
                obs = env.reset()
                obs_small = downsample_image(obs)
                state = np.stack([obs_small]*4, axis =0)

            else:
                state = next_state


        #Now play episodes and learning starts from here!!
        env = gym.envs.make("DemonAttack-v0")
        env = wrappers.Monitor(env, './', force= True)
        for i in range(num_episodes):
        
            total_t, episode_reward, duration, num_steps_in_episode, epsilon = play_one(env, total_t, experience_replay_buffer, model, target_model, gamma, batch_size , epsilon, epsilon_change, epsilon_min)
            episode_rewards[i] = episode_reward

            print("Episode:", i, " Duration:", duration, " Reward:", episode_reward)

            sys.stdout.flush()

c:\users\purvak\anaconda3\envs\py35\lib\site-packages\gym\envs\registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
c:\users\purvak\anaconda3\envs\py35\lib\site-packages\scipy\misc\pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if issubdtype(ts, int):
c:\users\purvak\anaconda3\envs\py35\lib\site-packages\scipy\misc\pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


Experience:  0
Experience:  1
Experience:  2
Experience:  3
Experience:  4
Experience:  5
Experience:  6
Experience:  7
Experience:  8
Experience:  9
Experience:  10
Experience:  11
Experience:  12
Experience:  13
Experience:  14
Experience:  15
Experience:  16
Experience:  17
Experience:  18
Experience:  19
Experience:  20
Experience:  21
Experience:  22
Experience:  23
Experience:  24
Experience:  25
Experience:  26
Experience:  27
Experience:  28
Experience:  29
Experience:  30
Experience:  31
Experience:  32
Experience:  33
Experience:  34
Experience:  35
Experience:  36
Experience:  37
Experience:  38
Experience:  39
Experience:  40
Experience:  41
Experience:  42
Experience:  43
Experience:  44
Experience:  45
Experience:  46
Experience:  47
Experience:  48
Experience:  49
Experience:  50
Experience:  51
Experience:  52
Experience:  53
Experience:  54
Experience:  55
Experience:  56
Experience:  57
Experience:  58
Experience:  59
Experience:  60
Experience:  61
Experience:  62
Ex

Experience:  500
Experience:  501
Experience:  502
Experience:  503
Experience:  504
Experience:  505
Experience:  506
Experience:  507
Experience:  508
Experience:  509
Experience:  510
Experience:  511
Experience:  512
Experience:  513
Experience:  514
Experience:  515
Experience:  516
Experience:  517
Experience:  518
Experience:  519
Experience:  520
Experience:  521
Experience:  522
Experience:  523
Experience:  524
Experience:  525
Experience:  526
Experience:  527
Experience:  528
Experience:  529
Experience:  530
Experience:  531
Experience:  532
Experience:  533
Experience:  534
Experience:  535
Experience:  536
Experience:  537
Experience:  538
Experience:  539
Experience:  540
Experience:  541
Experience:  542
Experience:  543
Experience:  544
Experience:  545
Experience:  546
Experience:  547
Experience:  548
Experience:  549
Experience:  550
Experience:  551
Experience:  552
Experience:  553
Experience:  554
Experience:  555
Experience:  556
Experience:  557
Experience:  5

Experience:  987
Experience:  988
Experience:  989
Experience:  990
Experience:  991
Experience:  992
Experience:  993
Experience:  994
Experience:  995
Experience:  996
Experience:  997
Experience:  998
Experience:  999
Experience:  1000
Experience:  1001
Experience:  1002
Experience:  1003
Experience:  1004
Experience:  1005
Experience:  1006
Experience:  1007
Experience:  1008
Experience:  1009
Experience:  1010
Experience:  1011
Experience:  1012
Experience:  1013
Experience:  1014
Experience:  1015
Experience:  1016
Experience:  1017
Experience:  1018
Experience:  1019
Experience:  1020
Experience:  1021
Experience:  1022
Experience:  1023
Experience:  1024
Experience:  1025
Experience:  1026
Experience:  1027
Experience:  1028
Experience:  1029
Experience:  1030
Experience:  1031
Experience:  1032
Experience:  1033
Experience:  1034
Experience:  1035
Experience:  1036
Experience:  1037
Experience:  1038
Experience:  1039
Experience:  1040
Experience:  1041
Experience:  1042
Exper

Experience:  1449
Experience:  1450
Experience:  1451
Experience:  1452
Experience:  1453
Experience:  1454
Experience:  1455
Experience:  1456
Experience:  1457
Experience:  1458
Experience:  1459
Experience:  1460
Experience:  1461
Experience:  1462
Experience:  1463
Experience:  1464
Experience:  1465
Experience:  1466
Experience:  1467
Experience:  1468
Experience:  1469
Experience:  1470
Experience:  1471
Experience:  1472
Experience:  1473
Experience:  1474
Experience:  1475
Experience:  1476
Experience:  1477
Experience:  1478
Experience:  1479
Experience:  1480
Experience:  1481
Experience:  1482
Experience:  1483
Experience:  1484
Experience:  1485
Experience:  1486
Experience:  1487
Experience:  1488
Experience:  1489
Experience:  1490
Experience:  1491
Experience:  1492
Experience:  1493
Experience:  1494
Experience:  1495
Experience:  1496
Experience:  1497
Experience:  1498
Experience:  1499
Experience:  1500
Experience:  1501
Experience:  1502
Experience:  1503
Experience

...

Experience:  48693
Experience:  48694
Experience:  48695
Experience:  48696
Experience:  48697
Experience:  48698
Experience:  48699
Experience:  48700
Experience:  48701
Experience:  48702
Experience:  48703
Experience:  48704
Experience:  48705
Experience:  48706
Experience:  48707
Experience:  48708
Experience:  48709
Experience:  48710
Experience:  48711
Experience:  48712
Experience:  48713
Experience:  48714
Experience:  48715
Experience:  48716
Experience:  48717
Experience:  48718
Experience:  48719
Experience:  48720
Experience:  48721
Experience:  48722
Experience:  48723
Experience:  48724
Experience:  48725
Experience:  48726
Experience:  48727
Experience:  48728
Experience:  48729
Experience:  48730
Experience:  48731
Experience:  48732
Experience:  48733
Experience:  48734
Experience:  48735
Experience:  48736
Experience:  48737
Experience:  48738
Experience:  48739
Experience:  48740
Experience:  48741
Experience:  48742
Experience:  48743
Experience:  48744
Experience: 

Experience:  49129
Experience:  49130
Experience:  49131
Experience:  49132
Experience:  49133
Experience:  49134
Experience:  49135
Experience:  49136
Experience:  49137
Experience:  49138
Experience:  49139
Experience:  49140
Experience:  49141
Experience:  49142
Experience:  49143
Experience:  49144
Experience:  49145
Experience:  49146
Experience:  49147
Experience:  49148
Experience:  49149
Experience:  49150
Experience:  49151
Experience:  49152
Experience:  49153
Experience:  49154
Experience:  49155
Experience:  49156
Experience:  49157
Experience:  49158
Experience:  49159
Experience:  49160
Experience:  49161
Experience:  49162
Experience:  49163
Experience:  49164
Experience:  49165
Experience:  49166
Experience:  49167
Experience:  49168
Experience:  49169
Experience:  49170
Experience:  49171
Experience:  49172
Experience:  49173
Experience:  49174
Experience:  49175
Experience:  49176
Experience:  49177
Experience:  49178
Experience:  49179
Experience:  49180
Experience: 

Experience:  49567
Experience:  49568
Experience:  49569
Experience:  49570
Experience:  49571
Experience:  49572
Experience:  49573
Experience:  49574
Experience:  49575
Experience:  49576
Experience:  49577
Experience:  49578
Experience:  49579
Experience:  49580
Experience:  49581
Experience:  49582
Experience:  49583
Experience:  49584
Experience:  49585
Experience:  49586
Experience:  49587
Experience:  49588
Experience:  49589
Experience:  49590
Experience:  49591
Experience:  49592
Experience:  49593
Experience:  49594
Experience:  49595
Experience:  49596
Experience:  49597
Experience:  49598
Experience:  49599
Experience:  49600
Experience:  49601
Experience:  49602
Experience:  49603
Experience:  49604
Experience:  49605
Experience:  49606
Experience:  49607
Experience:  49608
Experience:  49609
Experience:  49610
Experience:  49611
Experience:  49612
Experience:  49613
Experience:  49614
Experience:  49615
Experience:  49616
Experience:  49617
Experience:  49618
Experience: 

c:\users\purvak\anaconda3\envs\py35\lib\site-packages\gym\envs\registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


DependencyNotInstalled: Found neither the ffmpeg nor avconv executables. On OS X, you can install ffmpeg via `brew install ffmpeg`. On most Ubuntu variants, `sudo apt-get install ffmpeg` should do it. On Ubuntu 14.04, however, you'll need to install avconv with `sudo apt-get install libav-tools`.